# **Hypotheses**

### Objectives

* The objective is to understand the DataFrame in terms of its data regarding the mean,median, and standard deviation. These 3 aspects will give an understanding as to the middle value, average value, and deviation away from the mean or average value. Therefore, the spread of the data is in the dataset. If the standard deviation low then the data is likely to be clusterd, if it is high then there is likely to wide variability.

* The hypotheses used are based on assumptions and these 3 will be evaluated.

* Probability is concerned with number of outcomes and the likelihood of them happening.

#### Inputs

* I will use the ecommerce_transactions_cleaned.csv dataset.

#### Outputs

* The outputs will be from Chi, Welch's t-tests, and box plot visualisations.

#### Additional Comments

* These principles are important and form the foundation of data analysis because they provide the data analyst with insights, compare distributions and consistency, enable probablistic reasoning 



---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [2]:
import os
current_dir = os.getcwd()
current_dir

'c:\\Users\\rayaf\\OneDrive\\Documents\\global-store\\online_store\\jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [3]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [4]:
current_dir = os.getcwd()
current_dir

'c:\\Users\\rayaf\\OneDrive\\Documents\\global-store\\online_store'

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

##### Hypotheses 1

* Is the total purchase amount higher between March and August compared to November in the USA?

H₀ (null): Total purchase amount from March–August in the USA is equal to or less than that in November.

H₁ (alt): Total purchase amount from March–August in the USA is greater than in November.



##### Hypotheses 2


* Are Credit Card payments used significantly more than COD by customers?


H₀ (null): Payment methods are used equally — no preference observed.

H₁ (alt): Credit Card payments are used more frequently than COD.


##### Hypotheses 3

* The Sports category generates more sales than the Home & Kitchen category.

Hypothesis:
Null hypothesis (H0): Mean sales in Sports ≤ Mean sales in Home & Kitchen

Alternative hypothesis (H1): Mean sales in Sports > Mean sales in Home & Kitchen
